In [1]:
import os 
from openai import OpenAI
from func import get_user_watch_history, find_top_3_genre

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a movie recommender system that will compare user previous watch history and ratings.Based on the genre of those movies and the ratings that this particular user gave, please predict the next movie that this user is going to watch and predict the rating given by user"},
    {"role": "user", "content": "Below are the movie that I've watched and the corresponded ratings:5.0, Gladiator (2000), 4.0, Abyss, The (1989), 5.0, Canadian Bacon (1995), 2.0, Mummy, The (1999), 5.0, Star Wars: Episode IV - A New Hope (1977), 5.0  Messenger: The Story of Joan of Arc, The (1999), 5.0, Fugitive, The (1993). Please give me 5 movie suggestions based on my watch history and predict my ratings based on my preferences"}
  ]
)

print(completion.choices[0].message) 

ChatCompletionMessage(content='Based on your watch history and ratings, here are 5 movie suggestions for you:\n\n1. Braveheart (1995): This epic historical drama is similar to Gladiator and The Messenger: The Story of Joan of Arc, which you gave high ratings.\n\n2. Titanic (1997): This romantic disaster film might appeal to you, as it shares some similarities with The Abyss.\n\n3. Indiana Jones and the Raiders of the Lost Ark (1981): If you enjoyed adventure movies like The Mummy, you might like this classic action-adventure film.\n\n4. Star Wars: Episode V - The Empire Strikes Back (1980): Since you gave Star Wars: Episode IV - A New Hope a high rating, you might want to continue the saga with this highly acclaimed sequel.\n\n5. The Dark Knight (2008): This superhero film, known for its gripping story and strong performances, could be a good choice if you enjoyed The Fugitive.\n\nPlease note that these suggestions are based on the genre and ratings of the movies you have previously wa